In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import matplotlib.pyplot as plt
import datetime

Function that sends a SPARQL query to the OfficeKG endpoint, that returns the bindings of the query.

In [2]:
def query_ic_graph(query, bind, endpoint="https://data.interconnect.labs.vu.nl/sparql"):
    if not isinstance(bind, list):
        bind = [bind]
    sparql = SPARQLWrapper(
#         "http://130.37.53.36:6789/sparql"
        endpoint
    )
    sparql.setReturnFormat(JSON)

    sparql.setQuery(query)

    all_res = {}
    try:
        ret = sparql.queryAndConvert()
    #     print("ret:", ret)
        for b in bind:
            res = []
            for r in ret["results"]["bindings"]:
    #             print(r[bind]["value"])
                res.append(r[b]["value"])
            all_res[b] = res
    except Exception as e:
        print(e)
    if len(all_res.keys()) == 1:
        return all_res[bind[0]]
    return all_res

QUERY: Get all rooms and timestamps from devices measuring property x

In [6]:
q_get_all_room_time_for_property = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX saref: <https://saref.etsi.org/core/>
    PREFIX ic: <https://interconnectproject.eu/example/>
    PREFIX s4bldg: <https://saref.etsi.org/saref4bldg/>

SELECT DISTINCT ?room1 ?time1 ?dev1 WHERE {{
  ?dev1 saref:measuresProperty ?pred1 .
  ?pred1 a {} .
  ?dev1 s4bldg:isContainedIn ?room1 .
  ?dev1 saref:makesMeasurement ?meas1 .
  ?meas1 saref:relatesToProperty ?pred1 .
  ?meas1 saref:hasTimestamp ?time1 .
}}
"""

In [7]:
all_thermos_room_times = query_ic_graph(q_get_all_room_time_for_property.format("ic:thermostatHeatingSetpoint"), ["room1", "time1", "dev1"])


In [8]:
q_days_of_year = """
SELECT ?days ?daysLabel ?day_of_the_week WHERE 
{{
    ?months wdt:P361 {} .
    ?days wdt:P361 ?months .
    ?days wdt:P2894 ?day_of_the_week .
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}
"""


q_days_of_2022 = q_days_of_year.format("wd:Q49625")
q_days_of_2023 = q_days_of_year.format("wd:Q49622")

In [9]:
res22 = query_ic_graph(q_days_of_2022, ["days", "daysLabel", "day_of_the_week"], endpoint="https://query.wikidata.org/sparql")
res23 = query_ic_graph(q_days_of_2023, ["days", "daysLabel", "day_of_the_week"], endpoint="https://query.wikidata.org/sparql")


In [12]:
day_dict = {}

def create_day_dict(res, day_dict):
    for day, label, weekday in zip(res["days"], res["daysLabel"], res["day_of_the_week"]):
    #     print(day, label, weekday)
        dt = datetime.datetime.strptime(label, '%B %d, %Y')
#         print(dt.year, dt.month, dt.day)
        day_dict[" ".join([str(dt.year), str(dt.month), str(dt.day)])] = day
    return day_dict

day_dict = create_day_dict(res22, day_dict)
day_dict = create_day_dict(res23, day_dict)


In [13]:
def make_date_range(start, end):
    end = end+1
    date_range = []
    for i in range(start, end):
        i = str(i)
        if len(i) == 1:
            i = "0"+i
        date_range.append(str(i))
    return date_range


years = ["2022", "2023"]
months= make_date_range(1, 12)
months_length=[31,28,31,30,31,30,31,31,30,31,30,31]
days = make_date_range(1, 31)
hours = make_date_range(0, 23)

dates = []
for year in years:
    for month, month_length in zip(months,months_length):
        if year == "2022" and int(month) < 3: #dataset starts in March 2022
            continue
        if year == "2023" and int(month) > 1: #dataset ends in Februari 2023
            break
        for day in days:
            if int(day) > month_length:
                break
            for hour in hours:
                dates.append("ic:"+year+"-"+month+"-"+day+"_"+hour+"0000")


In [14]:
print((365-28)*24)
print(len(dates))

8088
8088


In [15]:
from rdflib import Graph, Namespace, RDF, RDFS, XSD, Literal, URIRef

In [16]:
ic = Namespace("https://interconnectproject.eu/example/")

g = Graph()
# g.bind("saref", "https://saref.etsi.org/core/")
g.bind("ic", "https://interconnectproject.eu/example/")
g.bind("wd", "http://www.wikidata.org/entity/")

for date in dates:
#     print(date)
    time = datetime.datetime.strptime(date, 'ic:%Y-%m-%d_%H%M%S')
#     print(day_dict[" ".join([str(time.year), str(time.month), str(time.day)])])
    g.add((URIRef(date), ic["atDate"], URIRef(day_dict[" ".join([str(time.year), str(time.month), str(time.day)])])))
#     print("ic:"+str(time.year)+"-"+str(time.month)+"-"+str(time.day)+_)
g.serialize(destination="wikidata_days.ttl", format='turtle')

<Graph identifier=N61caa1a279d44f48a362d02f3b560a8a (<class 'rdflib.graph.Graph'>)>

In [17]:
# print(all_thermos_room_times)
for time in all_thermos_room_times["time1"]:
    print(time)
    time = datetime.datetime.strptime(time, '%Y-%m-%dT%H:%M:%S')
    print(time.year, time.month, time.day)
    print(day_dict[" ".join([str(time.year), str(time.month), str(time.day)])])


2022-04-11T07:05:00
2022 4 11
http://www.wikidata.org/entity/Q69306396
2022-04-13T08:50:00
2022 4 13
http://www.wikidata.org/entity/Q69306398
2022-03-14T08:30:00
2022 3 14
http://www.wikidata.org/entity/Q69306365
2022-04-11T07:05:00
2022 4 11
http://www.wikidata.org/entity/Q69306396
2022-04-13T08:50:00
2022 4 13
http://www.wikidata.org/entity/Q69306398
2022-03-14T08:30:00
2022 3 14
http://www.wikidata.org/entity/Q69306365
2022-04-11T07:05:00
2022 4 11
http://www.wikidata.org/entity/Q69306396
2022-04-13T08:50:00
2022 4 13
http://www.wikidata.org/entity/Q69306398
2022-03-14T08:30:00
2022 3 14
http://www.wikidata.org/entity/Q69306365
2022-04-08T11:50:00
2022 4 8
http://www.wikidata.org/entity/Q69306393
2022-03-02T10:10:00
2022 3 2
http://www.wikidata.org/entity/Q69306353
2022-03-31T14:10:00
2022 3 31
http://www.wikidata.org/entity/Q69306385
2022-05-13T18:25:00
2022 5 13
http://www.wikidata.org/entity/Q69306428
2022-03-07T09:20:00
2022 3 7
http://www.wikidata.org/entity/Q69306358
2022-03-0

In [18]:
print(list(set(all_thermos_room_times["dev1"])))


['https://interconnectproject.eu/example/Zigbee_Thermostat_3', 'https://interconnectproject.eu/example/Zigbee_Thermostat_4', 'https://interconnectproject.eu/example/Zigbee_Thermostat_7', 'https://interconnectproject.eu/example/Zigbee_Thermostat_13', 'https://interconnectproject.eu/example/Zigbee_Thermostat_14', 'https://interconnectproject.eu/example/Zigbee_Thermostat_1', 'https://interconnectproject.eu/example/Zigbee_Thermostat_11', 'https://interconnectproject.eu/example/Zigbee_Thermostat_2', 'https://interconnectproject.eu/example/Zigbee_Thermostat_12', 'https://interconnectproject.eu/example/Zigbee_Thermostat_8', 'https://interconnectproject.eu/example/Zigbee_Thermostat_9']
